In [190]:
import pandas

def get_civ_involved_team(civ, df_team):
    df_auxw1 = df_team.loc[df_team['wCiv1'] == civ]
    df_auxw2 = df_team.loc[df_team['wCiv2'] == civ]
    df_auxw3 = df_team.loc[df_team['wCiv3'] == civ]
    df_auxw4 = df_team.loc[df_team['wCiv4'] == civ]
    df_auxl1 = df_team.loc[df_team['lCiv1'] == civ]
    df_auxl2 = df_team.loc[df_team['lCiv2'] == civ]
    df_auxl3 = df_team.loc[df_team['lCiv3'] == civ]
    df_auxl4 = df_team.loc[df_team['lCiv3'] == civ]
    return pandas.concat([df_auxw1,df_auxw2,df_auxw3,df_auxw4,df_auxl1,df_auxl2,df_auxl3,df_auxl4])

def get_civ_involved_x1(civ, df_x1):
    df_auxw1 = df_x1.loc[df_x1['wCiv'] == civ]
    df_auxl1 = df_x1.loc[df_x1['lCiv'] == civ]
    return pandas.concat([df_auxw1,df_auxl1])


def get_duration(civ, df):
    df_auxw1 = df.loc[df['duration'] == 'long']
    df_auxl1 = df.loc[df['duration'] != 'long']
    return pandas.concat([df_auxw1,df_auxl1])

In [191]:
import pandas

df_x1=pandas.read_json('x1_matches_dataset.json', dtype={'match_id': int, 'match_uuid': 'string', 'num_players': int,
                                                        'map_type' : int, 'visibility' : int, 'duration' : str,
                                                        'has_relevant_water' : bool, 'wCiv': str, 'lCiv' : str,
                                                        'average_rating' : float, 'average_rating_normalized': float })

df_team=pandas.read_json('team_matches_dataset.json', dtype={'match_id': int, 'match_uuid': 'string', 'num_players': int,
                                                             'map_type' : int, 'visibility' : int, 'duration' : str,
                                                             'has_relevant_water' : bool, 'wCiv1': str, 'lCiv1' : str,
                                                             'average_rating' : float, 'average_rating_normalized': float,
                                                             'wCiv2' : str, 'lCiv2' : str, 'wCiv3' : str, 'lCiv3' : str,
                                                             'wCiv4' : str, 'lCiv4' : str})

df_civ=pandas.read_json('Civilization_Dataset.json', dtype={int})
pandas.options.display.max_columns = None

In [192]:
df_x1['map_type']=df_x1['map_type'].astype('category')
df_x1['num_players']=df_x1['num_players'].astype('category')
df_x1['visibility']=df_x1['visibility'].astype('category')
df_x1['duration']=df_x1['duration'].astype('category')
df_x1['has_relevant_water']=df_x1['has_relevant_water'].astype('category')
df_x1['wCiv']=df_x1['wCiv'].astype('category')
df_x1['lCiv']=df_x1['lCiv'].astype('category')
df_x1.dtypes

map_type                     category
match_id                        int32
match_uuid                     object
num_players                  category
visibility                   category
duration                     category
has_relevant_water           category
wCiv                         category
lCiv                         category
average_rating                float64
average_rating_normalized     float64
dtype: object

In [193]:
df_team['map_type']=df_team['map_type'].astype('category')
df_team['num_players']=df_team['num_players'].astype('category')
df_team['visibility']=df_team['visibility'].astype('category')
df_team['duration']=df_team['duration'].astype('category')
df_team['has_relevant_water']=df_team['has_relevant_water'].astype('category')
df_team['wCiv1']=df_team['wCiv1'].astype('category')
df_team['wCiv2']=df_team['wCiv2'].astype('category')
df_team['wCiv3']=df_team['wCiv3'].astype('category')
df_team['wCiv4']=df_team['wCiv4'].astype('category')
df_team['lCiv1']=df_team['lCiv1'].astype('category')
df_team['lCiv2']=df_team['lCiv2'].astype('category')
df_team['lCiv3']=df_team['lCiv3'].astype('category')
df_team['lCiv4']=df_team['lCiv4'].astype('category')

df_team.dtypes

map_type                     category
match_id                        int32
match_uuid                     object
num_players                  category
visibility                   category
duration                     category
has_relevant_water           category
wCiv1                        category
wCiv2                        category
wCiv3                        category
wCiv4                        category
lCiv1                        category
lCiv2                        category
lCiv3                        category
lCiv4                        category
average_rating                float64
average_rating_normalized     float64
dtype: object

1º teste

Queremos fazer um teste de hipótese baseado nos possíveis upgrades das civilizações e sua interação com a duração da partida, em específico, queremos ver se civilizações com "trash units"(linguagem do jogo para unidades de baixíssimo custo) fortes, ou seja, com altos números de upgrade máximo para essas unidades costumam tornar partidas mais longas por sua capacidade de manter um fluxo de unidades fracas em campo e atrasar o andamento da partida.

Testaremos a hipotese utilizando o método binomial proportion ao comparar a proporção de partidas consideradas longas de todos os jogos rankeados com as partidas em que as civilizações em questão estão envolvidas, testaremos para p > p0 (se costumam ter partidas mais duradouras que as outras civilizações).

Logo abaixo, definímos as configurações das civilizações que se encaixam no padrão "trash units fortes"
(Perceba que Spearman e Skirmisher foram escolhidos, bem como seus upgrades mais influentes)

In [194]:
df_civ_aux=df_civ[df_civ['infantryarmorLine'] >= 3]
df_civ_aux=df_civ_aux[df_civ_aux['piercearmorLine'] >= 3]
df_civ_aux=df_civ_aux[df_civ_aux['skirmisherLine'] >= 2]
df_civ_aux=df_civ_aux[df_civ_aux['spearmanLine'] >= 3]
df_civ_aux

,militiaLine,spearmanLine,EaglescoutLine,has_supplies,has_squires,has_arson,archerLine,skirmisherLine,handcanoneerLine,cavalryarcherLine,has_thumbring,has_parthiantactics,scoutLine,knightLine,camelriderLine,battleelephantLine,steppelancerLine,has_bloodlines,has_husbandry,batteringramLine,mangonelLine,scorpionLine,bombardcannonLine,fireshipLine,demolitionraftLine,galleyLine,cannongalleonLine,has_drydock,has_shipwright,watchtowerLine,stonewallLine,piercearmorLine,pierceatkLine,meleeatkLine,cavalryarmorLine,infantryarmorLine,has_faith,has_redemption,has_illumination,has_atonement,has_blockprinting,has_herbalmedicine,has_theocracy,has_heresy,has_sanctity,has_fervor,has_masonry,has_architecture,has_fortifiedwall,has_bombardtower,has_siegeengineers,has_heatedshot,has_arrowslits,has_treadmillcrane,goldLine,stoneLine,woodLine,foodLine,has_guilds
Britons,3,3,0,1,1,1,3,2,0,2,0,0,2,2,0,0,0,0,1,2,2,2,0,3,3,3,1,1,1,3,2,3,2,2,3,3,1,0,1,0,1,1,1,0,1,1,1,1,1,0,1,1,1,0,2,1,2,1,1
Byzantines,3,3,0,1,1,1,3,2,1,2,1,0,3,3,2,0,0,0,1,3,2,1,1,3,3,3,2,1,1,3,2,3,2,1,3,3,1,1,1,1,1,0,1,1,1,1,0,0,1,1,0,0,1,0,2,2,2,2,1
Chinese,3,3,0,0,1,1,3,2,0,2,1,0,2,2,2,0,0,1,1,3,2,2,0,2,3,3,1,1,1,3,2,3,2,2,3,3,1,0,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,2,2,2,1,0
Japanese,3,3,0,1,1,1,3,2,1,2,1,1,2,2,0,0,0,1,1,2,2,2,0,3,2,3,2,1,1,3,2,3,2,2,2,3,1,1,1,1,1,1,1,0,1,1,1,0,1,0,1,0,1,1,2,1,2,1,0
Persians,1,3,0,1,1,1,2,2,1,2,1,1,3,3,2,0,0,1,1,3,2,2,1,3,3,3,2,1,0,2,1,3,1,2,3,3,1,0,0,0,1,1,1,0,0,1,1,1,0,0,0,1,0,0,2,2,2,2,1
Teutons,3,3,0,1,1,1,2,2,1,1,0,0,1,3,0,0,0,1,0,2,3,2,1,3,3,3,1,0,0,3,2,3,1,2,3,3,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,2,2,2,1
Koreans,3,3,0,1,1,1,3,2,1,2,1,0,3,2,0,0,0,0,1,2,3,1,1,3,0,3,1,1,1,3,2,3,2,1,2,3,1,0,0,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,2,2,2,1,1
Mayans,2,3,3,0,1,1,3,2,0,0,1,0,0,0,0,0,0,0,0,3,2,2,0,3,3,3,0,1,1,3,2,3,2,2,0,3,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,2,2,2,1
Spanish,3,3,0,1,1,1,1,2,1,2,1,0,3,3,0,0,0,1,1,3,2,1,1,3,3,3,2,1,1,3,2,3,2,2,3,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,2,2,1,1
Incas,3,3,3,1,1,1,3,2,0,0,1,0,0,0,0,0,0,0,0,3,2,2,0,3,2,3,0,1,1,3,2,3,2,2,0,3,1,1,1,0,1,1,1,1,1,0,1,0,1,0,1,1,1,1,2,2,1,2,1


In [195]:
df_civ_aux['long_matches_played']= 0
df_civ_aux['medium_and_short_matches_played']= 0
df_civ_aux=df_civ_aux[['long_matches_played', 'medium_and_short_matches_played']]

Aqui popularemos uma tabela com os jogos em que essas civilizações foram escolhidas

In [196]:
df_aux=get_civ_involved_x1('Britons', df_x1)

df_civ_aux.loc['Britons']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Britons']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_x1('Byzantines', df_x1)

df_civ_aux.loc['Byzantines']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Byzantines']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_x1('Chinese', df_x1)

df_civ_aux.loc['Chinese']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Chinese']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_x1('Japanese', df_x1)

df_civ_aux.loc['Japanese']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Japanese']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_x1('Persians', df_x1)

df_civ_aux.loc['Persians']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Persians']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_x1('Teutons', df_x1)

df_civ_aux.loc['Teutons']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Teutons']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_x1('Koreans', df_x1)

df_civ_aux.loc['Koreans']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Koreans']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_x1('Mayans', df_x1)

df_civ_aux.loc['Mayans']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Mayans']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_x1('Spanish', df_x1)

df_civ_aux.loc['Spanish']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Spanish']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_x1('Incas', df_x1)

df_civ_aux.loc['Incas']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Incas']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_x1('Slavs', df_x1)

df_civ_aux.loc['Slavs']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Slavs']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_x1('Ethiopians', df_x1)

df_civ_aux.loc['Ethiopians']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Ethiopians']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_x1('Portuguese', df_x1)

df_civ_aux.loc['Portuguese']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Portuguese']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_x1('Malay', df_x1)

df_civ_aux.loc['Malay']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Malay']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_x1('Vietnamese', df_x1)

df_civ_aux.loc['Vietnamese']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Vietnamese']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_x1('Cumans', df_x1)

df_civ_aux.loc['Cumans']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Cumans']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])
df_civ_aux

Tendo a tabela em mãos iniciaremos a análise dos dados:

As próximas 3 linhas definem a quantidade de jogos longos totais, a quantidade de jogos totais e a proporção de "sucesso", ou seja, a quantidade de jogos que são longos (resultando em aproximadamente 2,6%)

In [242]:
len(df_x1[df_x1['duration'] == 'long'])

528

In [243]:
len(df_x1[df_x1['duration'] != 'long'])

20353

In [244]:
base_propx1 = len(df_x1[df_x1['duration'] == 'long'])/len(df_x1[df_x1['duration'] != 'long'])
base_propx1

0.02594212155456198

Temos abaixo a proporção das durações das partidas das civilizações escolhidas acima, esses valores serão comparados com a proporção normal das partidas para checar se essas civilizações costumam ou não ter partidas longas

In [245]:
propx1 = df_civ_aux['long_matches_played']/(df_civ_aux['long_matches_played']+df_civ_aux['medium_and_short_matches_played'])

In [246]:
propx1

Britons       0.038190
Byzantines    0.042093
Chinese       0.026376
Japanese      0.030968
Persians      0.026270
Teutons       0.038835
Koreans       0.020501
Mayans        0.016819
Spanish       0.040865
Incas         0.028005
Slavs         0.029197
Ethiopians    0.018289
Portuguese    0.045929
Malay         0.015332
Vietnamese    0.029639
Cumans        0.035902
dtype: float64

Aqui compararemos as durações de cada civilização com as durações normais, True significa que mais do que 2,6% de suas partidas foram longas, enquanto que False significa que menos do que 2,6% de suas partidas foram longas.

In [247]:
long_match_tendencyx1 = propx1 > base_propx1

In [252]:
long_match_tendencyx1

Britons        True
Byzantines     True
Chinese        True
Japanese       True
Persians       True
Teutons        True
Koreans       False
Mayans        False
Spanish        True
Incas          True
Slavs          True
Ethiopians    False
Portuguese     True
Malay         False
Vietnamese     True
Cumans         True
dtype: bool

Faremos agora o teste para partidas de time

In [258]:
df_aux=get_civ_involved_team('Britons', df_team)

df_civ_aux.loc['Britons']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Britons']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_team('Byzantines', df_team)

df_civ_aux.loc['Byzantines']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Byzantines']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_team('Chinese', df_team)

df_civ_aux.loc['Chinese']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Chinese']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_team('Japanese', df_team)

df_civ_aux.loc['Japanese']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Japanese']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_team('Persians', df_team)

df_civ_aux.loc['Persians']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Persians']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_team('Teutons', df_team)

df_civ_aux.loc['Teutons']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Teutons']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_team('Koreans', df_team)

df_civ_aux.loc['Koreans']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Koreans']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_team('Mayans', df_team)

df_civ_aux.loc['Mayans']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Mayans']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_team('Spanish', df_team)

df_civ_aux.loc['Spanish']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Spanish']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_team('Incas', df_team)

df_civ_aux.loc['Incas']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Incas']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_team('Slavs', df_team)

df_civ_aux.loc['Slavs']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Slavs']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_team('Ethiopians', df_team)

df_civ_aux.loc['Ethiopians']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Ethiopians']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_team('Portuguese', df_team)

df_civ_aux.loc['Portuguese']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Portuguese']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_team('Malay', df_team)

df_civ_aux.loc['Malay']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Malay']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_team('Vietnamese', df_team)

df_civ_aux.loc['Vietnamese']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Vietnamese']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])

df_aux=get_civ_involved_team('Cumans', df_team)

df_civ_aux.loc['Cumans']['long_matches_played']=len(df_aux[df_aux['duration'] == 'long'])
df_civ_aux.loc['Cumans']['medium_and_short_matches_played']=len(df_aux[df_aux['duration'] != 'long'])
df_civ_aux

In [275]:
len(df_team[df_team['duration'] == 'long'])

299

In [276]:
len(df_team[df_team['duration'] != 'long'])

8943

In [277]:
base_propteam = len(df_team[df_team['duration'] == 'long'])/len(df_team[df_team['duration'] != 'long'])
base_propteam

0.0334339707033434

In [278]:
propteam = df_civ_aux['long_matches_played']/(df_civ_aux['long_matches_played']+df_civ_aux['medium_and_short_matches_played'])

In [279]:
propteam

Britons       0.038337
Byzantines    0.038204
Chinese       0.041667
Japanese      0.041860
Persians      0.044632
Teutons       0.044652
Koreans       0.032653
Mayans        0.030623
Spanish       0.034732
Incas         0.051414
Slavs         0.032807
Ethiopians    0.031721
Portuguese    0.021773
Malay         0.042139
Vietnamese    0.027113
Cumans        0.033735
dtype: float64

In [280]:
long_match_tendencyteam = propteam > base_propteam

In [281]:
long_match_tendencyteam

Britons        True
Byzantines     True
Chinese        True
Japanese       True
Persians       True
Teutons        True
Koreans       False
Mayans        False
Spanish        True
Incas          True
Slavs         False
Ethiopians    False
Portuguese    False
Malay          True
Vietnamese    False
Cumans         True
dtype: bool

In [288]:
long_match_tendencyx1

Britons        True
Byzantines     True
Chinese        True
Japanese       True
Persians       True
Teutons        True
Koreans       False
Mayans        False
Spanish        True
Incas          True
Slavs          True
Ethiopians    False
Portuguese     True
Malay         False
Vietnamese     True
Cumans         True
dtype: bool

Percebe-se que True se encontra na maioris das partidas x1 (partidas 1 contra 1), enquanto que para partidas de time, essa relação se manteve para a maior parte dos casos

Pode-se concluir que a maior parte das civilizações escolhidas dados os parâmetros realmente partcicipou de partidas mais longas que a média das outras 35 civilizações, até para a tabela de times esse resultado foi nítido, o que tem um significado mais influente porque até outras 7 civilizações podem estar envolvidas e alterar o curso da duração das partidas.